In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("reduction-op")
sc = SparkContext(conf=conf)

### Reduce
* `RDD.reduce(<task>)`
* 사용자가 지정하는 함수를 받아 여러 개의 값을 하나로 줄여줍니다.

In [2]:
from operator import add

In [3]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5]).reduce(add)
sample_rdd

15

#### 파티션에 따라 결과물이 달라집니다.
분산된 파티션들의 연산과 합치는 부분을 나눠서 생각해야 합니다.

```python
lambda x, y : (x * 2) + y
```

In [4]:
sc.parallelize([1, 2, 3, 4]).reduce(lambda x, y : (x * 2) + y)

26

|x|y|(x * 2) + y|
|:--:|:--:|:--:|
|1|2|(1 * 2) + 2 = 4|
|4|3|(4 * 2) + 3 = 11|
|11|4|(11 * 2) + 4 = 26|

In [5]:
# 파티션을 1개 지정 직접 하기
sc.parallelize([1, 2, 3, 4], 1).reduce(lambda x, y : (x * 2) + y)

26

In [6]:
# 파티션 2개로 직접 지정
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y : (x * 2) + y)

18

|p1_x|p1_y|(x * 2) + y|
|:--:|:--:|:--:|
|1|2|(1 * 2) + 2 = 4|

|p2_x|p2_y|(x * 2) + y|
|:--:|:--:|:--:|
|3|4|(3 * 2) + 4 = 10|

|p1_result|p2_result|(x * 2) + y|
|:--:|:--:|:--:|
|4|10|(4 * 2) + 10 = 18|

In [7]:
sc.parallelize([1, 2, 3, 4], 3).reduce(lambda x, y : (x * 2) + y)

18

**파티션에 데이터가 하나만 있으면 reduce가 안됩니다.**

|p1_x|p1_y|p1_result|
|:--:|:--:|:--:|
|1|0|1|

|p2_x|p2_y|p2_result|
|:--:|:--:|:--:|
|2|0|2|

|p3_x|p3_y|(x * 2) + y|
|:--:|:--:|:--:|
|3|4|(3 * 2) + 4 = 10|

|p1_result|p2_result|(x * 2) + y|
|:--:|:--:|:--:|
|3|4|(1 * 2) + 2 = 4|

|p1_p2_result|p3_result|(x * 2) + y|
|:--:|:--:|:--:|
|4|10|(4 * 2) + 10 = 18|

In [8]:
sc.parallelize([1, 2, 3, 4], 4).reduce(lambda x, y : (x * 2) + y)

26

### Fold
* `RDD.fold(zeroValue, <func>)`
* `reduce`와 비슷하지만, 시작값을 zeroValue에 넣어놓고 `reduce`할 수 있습니다.

In [9]:
rdd = sc.parallelize([2, 3, 4], 4)

print(rdd.reduce(lambda x, y : (x * y))) # 2 * 3 * 4
print(rdd.fold(1, lambda x, y : (x * y))) # 1(zeroValue * zeroValue) * 2(zeroValue * 2) * 3(zeroValue * 3) * 4(zeroValue * 4)

24
24


In [10]:
print(rdd.reduce(lambda x, y : x + y)) # 0 + 2 + 3 + 4
print(rdd.fold(1, lambda x, y: x + y)) # (1+1)+(1+2)+(1+3)+(1+4) = 14 # 빈 공간의 파티션에 숫자 1이 부여 되어서 1+1이 된다.

9
14


### GroupBy
* `RDD.groupBy(<func>)`
* 그룹핑 함수를 받아 reduction

In [12]:
rdd = sc.parallelize([1, 1, 2, 3, 5, 8, 10, 1,123,12,56,7,123,123,5,56,1236,8,8,45,657])
result = rdd.groupBy(lambda x : x % 2).collect()
sorted([(x, sorted(y)) for (x, y) in result])

[(0, [2, 8, 8, 8, 10, 12, 56, 56, 1236]),
 (1, [1, 1, 1, 3, 5, 5, 7, 45, 123, 123, 123, 657])]

### Aggregate ( Action )
* `RDD.aggregate(zeroValue, seqOp, combOp)`
    * `zeroValue` : 각 파티션에서 누적할 시작 값
    * `seqOp` : 타입 변경 함수. 파티션 내부에서 사용할 task
    * `combOp` : 합치는 함수. 파티션 끼리 사용할 task
* RDD 데이터 타입과 Action 결과 타입이 다를 경우 사용합니다.
* 파티션 단위의 연산 결과를 합치는 과정을 거칩니다.

In [13]:
rdd = sc.parallelize([1, 2, 3, 4], 2)

seqOp = (lambda x, y: (x[0] + y, x[1] + 1)) # 파티션 내의 연산
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1])) # 파티션의 모든 결과를 최종 연산

print(rdd.aggregate((0, 0), seqOp, combOp))

(10, 4)


#### 연산 과정
* partition 1 
    * x = `[1, 2]`, zeroValue=`[0, 0]`
    * `x[0]=0, x[1]=0` ( zeroValue )
    * (`x[0] + y = 0 + 1`, `x[1] + 1 = 0 + 1`) ==> (1, 1)
    * (`x[0] + y = 1 + 2`, `x[1] + 1 = 1 + 1`) ==> **(3, 2)**
* partition 2
    * x = `[3, 4]`, zeroValue=`[0, 0]`
    * `x[0]=0, x[1]=0` ( zeroValue )
    * (`x[0] + y = 0 + 3`, `x[1] + 1 = 0 + 1`) ==> (3, 1)
    * (`x[0] + y = 3 + 4`, `x[1] + 1 = 1 + 1`) ==> **(7, 2)**

`combOp`를 이용해 계산
* `x = (3, 2), y = (7, 2)`
* `(x[0] + y[0], x[1] + y[1])` = `(10, 4)`

In [14]:
sc.stop()